#Hadoop

![Hadoop Logo](https://github.com/pnavaro/big-data/blob/master/notebooks/images/hadoop.png?raw=1)

##Introdução

![https://github.com/veekaybee/data-lake-talk/](https://github.com/pnavaro/big-data/blob/master/notebooks/images/bigdata.png?raw=1)

- Framework para executar aplicativos em grandes clusters.
- O framework Hadoop fornece de forma transparente aos aplicativos confiabilidade e movimentação de dados.
- O Hadoop implementa o paradigma computacional chamado **Map/Reduce**, onde o aplicativo é dividido em muitos pequenos fragmentos de trabalho, cada um dos quais pode ser executado ou reexecutado em qualquer nó no cluster.
- Ele fornece um sistema de arquivos distribuído (HDFS) que armazena dados nos nós de computação, fornecendo largura de banda agregada muito alta em todo o cluster.
- Tanto o MapReduce quanto o **Hadoop Distributed File System** são projetados para que as falhas de nó sejam automaticamente tratadas pelo framework.

##Instalando o Java 8

Hadoop é um framework de processamento de dados baseado em Java.

O *OpenJDK* (versão *open source* do Java) é um ambiente de desenvolvimento para construir aplicativos, *applets* e componentes usando a linguagem de programação Java.

In [1]:
#Verificando a versão do Java instalada
!java -version

openjdk version "11.0.24" 2024-07-16
OpenJDK Runtime Environment (build 11.0.24+8-post-Ubuntu-1ubuntu322.04)
OpenJDK 64-Bit Server VM (build 11.0.24+8-post-Ubuntu-1ubuntu322.04, mixed mode, sharing)


Instalando o Java 8 para melhor compatibilidade com o Hadoop

In [2]:
#Instalando o java 8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

Alterando a versão padrão do Java no sistema

In [ ]:
#Alterando a versão do Java para usar como padrão (escolha a opção 2)
!update-alternatives --config java

In [ ]:
#Alterando a versão do Javac para usar como padrão (escolha a opção 2)
!update-alternatives --config javac

In [ ]:
#Alterando a versão do jps para usar como padrão (escolha a opção 2)
!update-alternatives --config jps

In [6]:
#Checking Java default version
!java -version

openjdk version "1.8.0_422"
OpenJDK Runtime Environment (build 1.8.0_422-8u422-b05-1~22.04-b05)
OpenJDK 64-Bit Server VM (build 25.422-b05, mixed mode)


Criando as variáveis ​​de ambiente relacionadas ao Java

**JAVA_HOME** é uma variável de ambiente do sistema operacional que aponta para o local do sistema de arquivos onde o JDK ou JRE foi instalado.

In [ ]:
#Encontrando o caminho Java padrão
!readlink -f /usr/bin/java | sed "s:bin/java::"

In [42]:
#Importando módulo os (sistema operacional)
import os
#Criando as variáveis ​​de ambiente
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["JRE_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64/jre"
os.environ["PATH"] += ":$JAVA_HOME/bin:$JRE_HOME/bin:$HADOOP_HOME/bin:$HADOOP_HOME/sbin"

## Instalando o Hadoop 3.2.3

In [9]:
#Download do Hadoop 3.2.3
!wget -q https://archive.apache.org/dist/hadoop/common/hadoop-3.2.3/hadoop-3.2.3.tar.gz

In [10]:
#Descompactando o arquivo
!sudo tar -xzf hadoop-3.2.3.tar.gz

#Removendo o arquivo compactado
!rm hadoop-3.2.3.tar.gz

O local padrão para instalar o Hadoop é:

*   /usr/local
*   /opt

In [11]:
#Copiando os arquivos do hadoop para --> /usr/local/
!cp -r hadoop-3.2.3/ /usr/local/
#-r copia diretórios recursivamente

In [12]:
#Explorando o diretório --> hadoop-3.2.3/etc/hadoop
!ls /usr/local/hadoop-3.2.3/etc/hadoop
#Podemos ver vários arquivos de configuração do hadoop

capacity-scheduler.xml		  httpfs-log4j.properties     mapred-site.xml
configuration.xsl		  httpfs-signature.secret     shellprofile.d
container-executor.cfg		  httpfs-site.xml	      ssl-client.xml.example
core-site.xml			  kms-acls.xml		      ssl-server.xml.example
hadoop-env.cmd			  kms-env.sh		      user_ec_policies.xml.template
hadoop-env.sh			  kms-log4j.properties	      workers
hadoop-metrics2.properties	  kms-site.xml		      yarn-env.cmd
hadoop-policy.xml		  log4j.properties	      yarn-env.sh
hadoop-user-functions.sh.example  mapred-env.cmd	      yarnservice-log4j.properties
hdfs-site.xml			  mapred-env.sh		      yarn-site.xml
httpfs-env.sh			  mapred-queues.xml.template


Precisamos configurar algumas coisas antes de executar o Hadoop. Ou seja, precisamos adicionar ou modificar alguns parâmetros nesses arquivos de configuração para operar o Hadoop em qualquer modo que quisermos.

Configurando o arquivo **hadoop-env.sh**

**hadoop-env.sh** é um *script bash* que contém variáveis ​​de ambiente que são usadas nos scripts para executar o Hadoop

In [ ]:
#Visualizando o conteudo do aquivo --> hadoop-env.sh
!cat /usr/local/hadoop-3.2.3/etc/hadoop/hadoop-env.sh

A única variável de ambiente necessária é **JAVA_HOME**. Todas as outras são opcionais.

Para especificar a variável JAVA_HOME em hadoop-env.sh, precisamos descomentar a linha export e atualizá-la com o diretório real.

Neste caso, deve ficar assim:

`export JAVA_HOME=/usr/lib/jvm/java-8-openjdk-amd64`

In [14]:
#Adicionando o diretório do JAVA_HOME ao arquivo --> hadoop-env.sh
!sed -i '/export JAVA_HOME=/a export JAVA_HOME=\/usr\/lib\/jvm\/java-8-openjdk-amd64' /usr/local/hadoop-3.2.3/etc/hadoop/hadoop-env.sh

Criando uma variável de ambiente que aponta para o diretório de instalação do Hadoop

In [15]:
#Variável de ambiente para o diretório do hadoop
os.environ["HADOOP_HOME"] = "/usr/local/hadoop-3.2.3"

Configurando arquivos XML

A maioria das configurações do Hadoop está contida em arquivos de configuração XML. Esses arquivos também são conhecidos como **recursos**.

Eles têm a seguinte estrutura:


```
<configuration>
...
  <property>
    <name>...</name>
    <value>...</value>
    <description>...</description>
  </property>
...
</configuration>
```



A configuração do Hadoop é conduzida por dois tipos distintos de arquivos de configuração XLM:

1. **Default** (somente leitura): *core-default.xml, hdfs-default.xml, mapred-default.xml, yarn-default.xml*. Esses arquivos nunca devem ser modificados.
2. Arquivos de configuração **Site specific**: *core-site.xml, hdfs-site.xml, mapred-site.xml, yarn-site.xml*. Esses arquivos são carregados do caminho da classe e seus valores são usados ​​para substituir os valores correspondentes das propriedades nos arquivos de configuração padrão correspondentes.

In [ ]:
#Explorando os arquivos do diretório --> hadoop-3.2.3/etc/hadoop xml
!ls $HADOOP_HOME/etc/hadoop/*.xml

Cada componente no Hadoop é configurado usando um arquivo xml

* core-site.xml: propriedades comuns
* hdfs-site.xml: propriedades do HDFS
* mapred-site.xml: propriedades do MapReduce
* yarn-site.xml: propriedades do YARN

Ao configurar esses arquivos xml adequadamente, o Hadoop pode ser executado em um dos três modos.

In [ ]:
#Conteudo do arquivo --> core-site.xml file
!cat $HADOOP_HOME/etc/hadoop/core-site.xml

Como podemos ver, nenhuma propriedade foi definida. Elas estão vazias por padrão. Então não há nada para sobrescrever e o Hadoop funcionará com as propriedades padrão.

## Executando o Hadoop

Com as propriedades de configuração padrão, o Hadoop é executado em um modo autônomo (*modo não distribuído*). Ou seja, o modo autônomo (também conhecido como modo local) é o modo padrão para o Hadoop.

O sistema de arquivos e o executor de tarefas MapReduce local serão utilizados.

O comando para executar um programa *Hadoop mapreduce* escrito em Java é:

`$HADOOP_HOME/bin/hadoop jar <jar>`

**jar** é uma ferramenta Java que empacota (e compacta) um conjunto de classes/pacotes em um único arquivo.

A instalação padrão já possui vários programas de exemplo do MapReduce que podemos usar.

In [18]:
#Explorando as ferramentas de mapreduce
!ls $HADOOP_HOME/share/hadoop/mapreduce/*.jar

/usr/local/hadoop-3.2.3/share/hadoop/mapreduce/hadoop-mapreduce-client-app-3.2.3.jar
/usr/local/hadoop-3.2.3/share/hadoop/mapreduce/hadoop-mapreduce-client-common-3.2.3.jar
/usr/local/hadoop-3.2.3/share/hadoop/mapreduce/hadoop-mapreduce-client-core-3.2.3.jar
/usr/local/hadoop-3.2.3/share/hadoop/mapreduce/hadoop-mapreduce-client-hs-3.2.3.jar
/usr/local/hadoop-3.2.3/share/hadoop/mapreduce/hadoop-mapreduce-client-hs-plugins-3.2.3.jar
/usr/local/hadoop-3.2.3/share/hadoop/mapreduce/hadoop-mapreduce-client-jobclient-3.2.3.jar
/usr/local/hadoop-3.2.3/share/hadoop/mapreduce/hadoop-mapreduce-client-jobclient-3.2.3-tests.jar
/usr/local/hadoop-3.2.3/share/hadoop/mapreduce/hadoop-mapreduce-client-nativetask-3.2.3.jar
/usr/local/hadoop-3.2.3/share/hadoop/mapreduce/hadoop-mapreduce-client-shuffle-3.2.3.jar
/usr/local/hadoop-3.2.3/share/hadoop/mapreduce/hadoop-mapreduce-client-uploader-3.2.3.jar
/usr/local/hadoop-3.2.3/share/hadoop/mapreduce/hadoop-mapreduce-examples-3.2.3.jar


Mapreduce examples

In [ ]:
#Explorando os exemplos de programas disponíveis
!$HADOOP_HOME/bin/hadoop jar $HADOOP_HOME/share/hadoop/mapreduce/hadoop-mapreduce-examples-3.2.3.jar

wordcount

Como a descrição diz, o ***wordcount*** é um programa de mapeamento/redução (*map/reduce*) que conta as palavras nos arquivos de entrada.

In [ ]:
#Uso do programa MapReduce de wordcount (contagem de palavras)
!$HADOOP_HOME/bin/hadoop jar $HADOOP_HOME/share/hadoop/mapreduce/hadoop-mapreduce-examples-3.2.3.jar wordcount

Os parâmetros são um diretório de entrada onde o texto a ser analisado é alocado e um diretório de saída onde o programa irá alocar sua saída.

In [44]:
#Baixando um exemplo de texto para usar como entrada
!wget -q https://www.mirrorservice.org/sites/ftp.ibiblio.org/pub/docs/books/gutenberg/1/0/101/101.txt

In [ ]:
#Executando o programa MapReduce wordcount
#O diretório de saída será criado automaticamente
!$HADOOP_HOME/bin/hadoop jar $HADOOP_HOME/share/hadoop/mapreduce/hadoop-mapreduce-examples-3.2.3.jar wordcount /content/101.txt /content/saida

In [ ]:
#Explorando o diretório de saída
#part-r-00000 contém o resultado
!ls /content/saida

In [ ]:
#Imprimindo o conteúdo do arquivo
!cat /content/saida/part-r-00000

In [ ]:
#Imprimindo as primeiras 50 linhas
!head -100 /content/saida/part-r-00000